<a href="https://stella-project.org/"><img align="right" width="100" src="doc/img/logo-st.JPG" /></a>

# STELLA Demo
This interactive demo guide walks you through the [STELLA](https://stella-project.org/) infrastructure using the STEALLA [demo.yml](https://github.com/stella-project/stella-app/blob/master/demo.yml) docker-compose stack. It is designed to give a simple but comprehensive overview of the functionalities of the STELLA Living Lab infrastructure.

<br>

**Table of content**
1. Setup
2. STELLA app
  - 2.1 Indexing documents
  - 2.2 Querying results
  - 2.3 Saving feedback
3. STELLA server

## 1. Setup
---
For an in depth setup guide please visit the [full documentation](https://github.com/stella-project/stella-documentation/wiki/Setup-guide-(local)). For this guide, the [demo.yml](https://github.com/stella-project/stella-app/blob/master/demo.yml) compose-stack is used. 

Before starting, please make sure:
1. `docker` and `docker-compose` are installed,
2. the repositorys [stella-app](https://github.com/stella-project/stella-app) and [stella-server](https://github.com/stella-project/stella-server) are cloned and
3. you downloaded at least some of the [data](https://th-koeln.sciebo.de/s/OBm0NLEwz1RYl9N?path=%2F) to the `/data` directory.

If these requirements are satisfied, at most, minimal adjustments should be needed. While designed as a distributed system, we will set up all components on one machine, using a single docker-compose stack. Any changes, if needed, take place in the [demo.yml](https://github.com/stella-project/stella-app/blob/master/demo.yml) file. If not freely available on your machine, make sure to change the ports `8000` and `8080`. 
Since indexing the data can be performance-intensive, depending on the amount of data, experimental systems and your system specifications, it can be necessary to disable the `BULK_INDEX` option.

After that, you should be ready to start your system. This may take a while, so please be patient.

In [ ]:
!docker-compose -f demo.yml up -d

<br>

## 2. STELLA app:
---

The STELLA app connects the search user interface, like [LIVIVO](https://www.livivo.de), [Gesis](https://www.gesis.org/en/home) or your own,  with the experimental ranking systems and is responsible for interleaving the result pages. For this demo we installed the ranking systems `livivo_rank_precom`, `livivo_rank_pyserini`, `gesis_rec_precom` and `gesis_rec_pyserini`, just to cover all use-cases. These four ranking systems, two baseline systems and two experimental systems are specified in the [demo.yml](https://github.com/stella-project/stella-app/blob/master/demo.yml) file. To learn how to develop your own ranking system or using a simple pre-computed run please visit the full [documentation](https://github.com/stella-project/stella-documentation/wiki).

For interaction and communication between systems different endpoints are available and described in the following:

In [16]:
STELLA_URL = 'http://0.0.0.0:8080'
STELLA_APP_API = 'http://0.0.0.0:8080/stella/api/v1/'


### 2.1 Indexing documents:
If `BULK_INDEX` is disabled while creating the docker-compose stack, systems can be indexed either through the [stella-app web ui](http://0.0.0.0:8080) or the stella-app endpoint.

**1. Web UI:**

You can reach the stella-app web UI via [http://0.0.0.0:8080/](http://0.0.0.0:8080/) and trigger indexing for specific systems.

**2. Endpoint:**
- `/index/<system_name>` let you index a specific system.

In [17]:
import requests

In [15]:
SYSTEM_NAME = 'livivo_rank_pyserini'

r = requests.get(STELLA_URL + '/index/' + SYSTEM_NAME)
print(r.text)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>



- `/index/bulk` let you trigger the full bulk index.

In [18]:
r = requests.get(STELLA_URL + '/index/bulk')
print(r.text)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>



Indexing may take a while. If it`s done, you are ready for your first search.

### 2.2 Querying results:
For querying interleaved results you can use two endpoints, depending on the system type.

#### GET ranking
will return an interleaved ranking result, given a query string, from an experimental ranking system and the base system.

In [19]:
query = "influenza"

In [30]:
r = requests.get(STELLA_APP_API + "ranking?query=" + query)
ranking = r.json()

#### GET recommendation
returns interleaved recommendations, given the id of a seed document, from an experimental system and the base system.

In [26]:
itemid = "gesis-ssoar-13114"

In [31]:
r = requests.get(STELLA_APP_API + "recommendation/datasets?itemid=" + itemid)
recommendation = r.json()

### 2.3 Saving feedback:

These result lists will be clicked by the users and then returned to the app, including the feedback data, using the feedback endpoints. Multiple feedbacks can be combined into sessions.

For this demo we will simulate random user feedbacks by this function:

In [45]:
import datetime
import random
import json

def simulate(req_json):
    click_dict = req_json.get('body')
    session_start_date = datetime.datetime.now()
    session_end_date = session_start_date + datetime.timedelta(0, random.randint(10, 3000))
    random_clicks = random.sample(range(1, len(click_dict)), random.randint(1, 3))
    random.sample(range(1, 10), random.randint(1, 9))
    for key, val in click_dict.items():
        if int(key) in random_clicks:
            click_dict.update({key: {'docid': req_json.get('body').get(key).get('docid'),
                                     'type': req_json.get('body').get(key).get('type'),
                                     'clicked': True,
                                     'date': session_start_date.strftime("%Y-%m-%d %H:%M:%S")}})
        else:
            click_dict.update({key: {'docid': req_json.get('body').get(key).get('docid'),
                                     'type': req_json.get('body').get(key).get('type'),
                                     'clicked': False,
                                     'date': None}})
    return {
        'start': session_start_date.strftime("%Y-%m-%d %H:%M:%S"),
        'end': session_end_date.strftime("%Y-%m-%d %H:%M:%S"),
        'interleave': True,
        'clicks': json.dumps(click_dict)
    }

#### POST ranking feedback:

In [42]:
rec_id = ranking.get('header').get('rid')

In [44]:
payload = simulate(ranking)
r_post = requests.post(STELLA_APP_API + "ranking/" + str(rec_id) + "/feedback", data=payload)
print(r_post.status_code)

201


#### POST recommendation feedback:

In [ ]:
payload = simulate(recommendation)
r_post = requests.post(STELLA_APP_API + "recommendation/" + str(rec_id) + "/feedback", data=payload)
print(r_post.status_code)

All feedbacks will be regularly transfered to the STELLA server using the stella server api described in the next part.

<br>

## 3. STELLA server:
---
The STELLA server not just saves all user feedback but lets you administrate all systems. Furthermore, it is the main interaction point for participants, submitting and monitoring their systems.

In [4]:
STELLA_SERVER_API = 'http://0.0.0.0:8000/stella/api/v1'

In [5]:
# token and user rolls

In [6]:
# für demo session erstellen

In [7]:
# POST session

In [8]:
# POST feedback

In [9]:
# 

In [ ]:
# GET Session

In [10]:
# GET Feedback

In [ ]:
#

In [ ]:
# GET sessions from site

In [ ]:
# GET Systems of participant

In [ ]:
# GET full results for user/site